In [1]:
pip install -U gym

Note: you may need to restart the kernel to use updated packages.
  Created wheel for gym: filename=gym-0.18.0-py3-none-any.whl size=1656455 sha256=cfd7c5ad1fb93cb86f6bd65acf591f02746f7ae5d722b114ea95e1bbe625404d
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\d8\e7\68\a3f0f1b5831c9321d7523f6fd4e0d3f83f2705a1cbd5daaa79
Successfully built gym
  Attempting uninstall: Pillow
    Found existing installation: Pillow 8.0.1
    Uninstalling Pillow-8.0.1:
      Successfully uninstalled Pillow-8.0.1


In [4]:
import gym
env=gym.make("CartPole-v1")
obs=env.reset()


In [5]:
img=env.render(mode='rgb_array')
img.shape

(400, 600, 3)

In [6]:
env.action_space

Discrete(2)

In [9]:
action=1
obs,reward,done,info=env.step(action)
obs

array([ 0.0524482 ,  0.36415552,  0.02356664, -0.5649698 ])

In [10]:
reward

1.0

In [11]:
done

False

In [12]:
info

{}

In [14]:
def basicpo(obs):
    angle=obs[2]
    return 0 if angle<0 else 1
total=[]
for epi in range(500):
    epireward=0
    obs=env.reset()
    for step in range(200):
        action=basicpo(obs)
        obs,reward,done,info=env.step(action)
        epireward+=reward
        if done:
            break
    total.append(epireward)

In [15]:
import numpy as np
np.mean(total),np.std(total),np.min(total),np.max(total)

(42.818, 8.727249051104248, 24.0, 68.0)

In [16]:
env.close()

In [18]:
obs

array([-0.16153228, -0.95981935,  0.23536147,  1.24079936])

In [58]:
import tensorflow as tf
from tensorflow import keras

ninput=4

model=keras.models.Sequential([
    keras.layers.Dense(5,activation='elu',input_shape=[ninput]),
    keras.layers.Dense(1,activation='sigmoid')
])

In [56]:
def play_one_step(env, obs, model, loss_fn):
    with tf.GradientTape() as tape:
        left_proba = model(obs[np.newaxis])
        action = (tf.random.uniform([1, 1]) > left_proba)
        y_target = tf.constant([[1.]]) - tf.cast(action, tf.float32)
        loss = tf.reduce_mean(loss_fn(y_target, left_proba))
    grads = tape.gradient(loss, model.trainable_variables)
    obs, reward, done, info = env.step(int(action[0, 0].numpy()))
    return obs, reward, done, grads

In [36]:
def play_multiple_episodes(env, n_episodes, n_max_steps, model, loss_fn):
    all_rewards = []
    all_grads = []
    for episode in range(n_episodes):
        current_rewards = []
        current_grads = []
        obs = env.reset()
        for step in range(n_max_steps):
            obs, reward, done, grads = play_one_step(env, obs, model, loss_fn)
            current_rewards.append(reward)
            current_grads.append(grads)
            if done:
                break
        all_rewards.append(current_rewards)
        all_grads.append(current_grads)
    return all_rewards, all_grads

In [37]:
def discount_reward(reward,discount_factor):
    discount=np.array(reward)
    for step in range(len(reward)-2,-1,-1):
        discount[step]+=discount[step+1]*discount_factor
    return discount

def discount_normalize(all_rewards,discount_factor):
    all_discount_rewards=[discount_reward(rewards,discount_factor)for rewards in all_rewards]
    flat_rewards=np.concatenate(all_discount_rewards)
    reward_mean=flat_rewards.mean()
    reward_std=flat_rewards.std()
    return [(discount_reward-reward_mean)/reward_std for discount_reward in all_discount_rewards]

In [38]:
discount_reward([10,0,-50],discount_factor=0.8)

array([-22, -40, -50])

In [39]:
discount_normalize([[10,0,-50],[10,20]],discount_factor=0.8)

[array([-0.28435071, -0.86597718, -1.18910299]),
 array([1.26665318, 1.0727777 ])]

In [46]:
n_iterations = 150
n_episodes_per_update = 10
n_max_steps = 200
discount_rate = 0.95

In [47]:
optimizer=keras.optimizers.Adam(lr=0.01)
loss_fn=keras.losses.binary_crossentropy


In [ ]:
env=gym.make('CartPole-v1')
for iterr in  range(n_iter):
    all_rewards,all_grads=play_multiple_episodes(env,n_epiperupdate,n_max_steps,model,loss_fn)
    all_final_rewards=discount_normalize(all_rewards,discount_factor)
    
    all_mean_grads=[]
    for var_index  in range(len(model.trainable_variables)):
        mean_grads=tf.reduce_mean(
            [final_reward * all_grads[epiindex][step][var_index] 
             for epiindex,final_reward in enumerate(all_final_rewards)
                 for step,final_reward in enumerate(final_reward)],axis=0)
        all_mean_grads.append(mean_grads)
    optimizer.apply_gradients(zip(all_mean_grads,model.trainable_variables))
env.close()